<a href="https://colab.research.google.com/github/taehyeonk/pre-onboarding_project/blob/jina/220315_sub3_roberta_large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence_transformers

     |████████████████████████████████| 79 kB 5.0 MB/s 
     |████████████████████████████████| 3.8 MB 64.8 MB/s 
     |████████████████████████████████| 1.2 MB 4.4 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 81.7 MB/s 
     |████████████████████████████████| 6.5 MB 80.4 MB/s 
     |████████████████████████████████| 895 kB 54.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=510a502a2efa3dbae8e7a92a2416a7b525e37ce617ac29866de4411242399dd5
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import argparse
import logging
import math
import os
import random
from datetime import datetime
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression

from sklearn.manifold import TSNE
from matplotlib import pyplot as plt

from sentence_transformers import SentenceTransformer, LoggingHandler, models, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, BinaryClassificationEvaluator
from torch.utils.data import DataLoader

import json
from sentence_transformers.readers import InputExample

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Colab Notebooks/klue-sts-v1.1

/content/drive/MyDrive/Colab Notebooks/klue-sts-v1.1


In [5]:
def load_kor_sts_samples(filename):
    samples = []
    sc = []
    sc2 = []
    st = []
    st2 = []
    gu =[]
    gu2 =[]
    temp_df=pd.DataFrame()
    with open(filename,'r') as f:
        json_data = json.load(f)
    for row in json_data:
        score = float(row['labels']['binary-label'])
        sc.append(score)
        st.append(row['sentence1'])
        gu.append(row['guid'])
        sc2.append(score)
        st2.append(row['sentence2'])
        gu2.append(row['guid'])
        samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score, guid=row['guid']))
    temp_df['guid']=pd.concat([pd.Series(gu),pd.Series(gu2)],axis=0,ignore_index=True)
    temp_df['sentence']=pd.concat([pd.Series(st),pd.Series(st2)],axis=0,ignore_index=True)
    temp_df['label']=pd.concat([pd.Series(sc),pd.Series(sc2)],axis=0,ignore_index=True)
    return samples, temp_df

In [6]:
logging.info("Read KorSTS train/dev dataset")

train_file = './klue-sts-v1.1_train.json'
train_samples, xy_data = load_kor_sts_samples(train_file)

test_file = './klue-sts-v1.1_dev.json'
test_samples, xy_data_eval=load_kor_sts_samples(test_file)

n_train = int(0.8 * len(train_samples))
valid_samples = train_samples[n_train:]
train_samples = train_samples[:n_train]

In [8]:
#@title Setfit
st_model='klue/roberta-large' #@param ['klue/roberta-large','klue/roberta-base','klue/bert-base','skt/kobert-base-v1'] 
num_itr = 5 #@param ["1", "2", "3", "4", "5", "10"] {type:"raw"}
plot2d_checkbox = False #@param {type: 'boolean'}

In [9]:
# Parse command line arguments
parser = argparse.ArgumentParser()
parser.add_argument("--model_name_or_path", type=str, default = st_model)
parser.add_argument("--max_seq_length", type=int, default=256)
parser.add_argument("--batch_size", type=int, default=16)
parser.add_argument("--num_epochs", type=int, default=5)
parser.add_argument("--output_dir", type=str, default="output")
parser.add_argument("--output_prefix", type=str, default="kor_sts_")
parser.add_argument("--seed", type=int, default=777)
args = parser.parse_args("")

# Fix random seed
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

# Configure logger
logging.basicConfig(
    format='%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S', level=logging.INFO, handlers=[LoggingHandler()]
)

# Read the dataset
model_save_path = os.path.join(
    args.output_dir,
    args.output_prefix + args.model_name_or_path.replace("/", "-") + '-' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)

In [11]:
# Define SentenceTransformer model
word_embedding_model = models.Transformer(args.model_name_or_path, max_seq_length=args.max_seq_length)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model], device = 'cuda')

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=args.batch_size)
valid_evaluator = BinaryClassificationEvaluator.from_input_examples(valid_samples, batch_size=args.batch_size,name='sts-valid')
warmup_steps = math.ceil(len(train_dataloader) * args.num_epochs * 0.1) 
# S-BERT adaptation 

train_loss = losses.ContrastiveLoss(model=model)
model.fit(train_objectives=[(train_dataloader, train_loss)],evaluation_steps=1000,
          evaluator=valid_evaluator,epochs=4,show_progress_bar = True,warmup_steps=warmup_steps,
          output_path=model_save_path)

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/584 [00:00<?, ?it/s]

Iteration:   0%|          | 0/584 [00:00<?, ?it/s]

Iteration:   0%|          | 0/584 [00:00<?, ?it/s]

Iteration:   0%|          | 0/584 [00:00<?, ?it/s]

In [12]:
test_evaluator_1 = BinaryClassificationEvaluator.from_input_examples(test_samples, name='sts-test')
test_evaluator_1(model, output_path=model_save_path)

0.9152136729976093